**Load secrets and connect to database**

In [1]:
#Get plaintext secrets

from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('secrets.cfg')
neo4j_useraccount = parser.get('my_data', 'neo4j_useraccount')
neo4j_password = parser.get('my_data', 'neo4j_password')


In [2]:
from graphdatascience import GraphDataScience

host = "bolt://172.17.0.3:7687"
user = neo4j_useraccount
password= neo4j_password

gds = GraphDataScience(host, auth=(user, password))

In [3]:
%%time
#Create constraint for process node, massivly speeds up import
unique_originalFileName = gds.run_cypher("""
CREATE CONSTRAINT Process FOR (process:Process) REQUIRE process.ProcessGuid IS UNIQUE
""")

CPU times: user 1.83 ms, sys: 1.29 ms, total: 3.12 ms
Wall time: 73.1 ms


**Get list of files to be imported**

In [4]:
import os
#Gets all process node files and puts them in a variable
def get_process_node_files(directory):
    process_node_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_process_nodes.csv'):
            process_node_files.append(filename)
    return process_node_files

process_node_result=get_process_node_files("/home/jovyan/work/neo4j/import/20-23/")
print(process_node_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_process_nodes.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_process_nodes.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_process_nodes.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_process_nodes.csv', 'AIA-201-225.ecar-last_process_nodes.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_process_nodes.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_process_nodes.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_process_nodes.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_process_nodes.csv', 'AIA-501-525.ecar-last_process_nodes.csv']


In [5]:
import os
#Gets all process relation files and puts them in a variable
def get_process_relation_files(directory):
    process_relation_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_process_relations.csv'):
            process_relation_files.append(filename)
    return process_relation_files

process_relation_result=get_process_relation_files("/home/jovyan/work/neo4j/import/20-23/")
print(process_relation_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_process_relations.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_process_relations.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_process_relations.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_process_relations.csv', 'AIA-201-225.ecar-last_process_relations.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_process_relations.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_process_relations.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_process_relations.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_process_relations.csv', 'AIA-501-525.ecar-last_process_relations.csv']


In [6]:
import os
#Gets all process node files and puts them in a variable
def get_file_node_files(directory):
    file_nodes_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_file_nodes.csv'):
            file_nodes_files.append(filename)
    return file_nodes_files

file_node_result=get_file_node_files("/home/jovyan/work/neo4j/import/20-23/")
print(file_node_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_file_nodes.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_file_nodes.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_file_nodes.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_file_nodes.csv', 'AIA-201-225.ecar-last_file_nodes.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_file_nodes.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_file_nodes.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_file_nodes.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_file_nodes.csv', 'AIA-501-525.ecar-last_file_nodes.csv']


In [7]:
import os
#Gets all process node files and puts them in a variable
def get_file_relations_files(directory):
    file_relations_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_file_relations.csv'):
            file_relations_files.append(filename)
    return file_relations_files

file_relations_result=get_file_relations_files("/home/jovyan/work/neo4j/import/20-23/")
print(file_relations_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_file_relations.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_file_relations.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_file_relations.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_file_relations.csv', 'AIA-201-225.ecar-last_file_relations.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_file_relations.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_file_relations.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_file_relations.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_file_relations.csv', 'AIA-501-525.ecar-last_file_relations.csv']


In [10]:
import os
#Gets all flow node files and puts them in a variable
def get_network_node_files(directory):
    network_node_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_network_node.csv'):
             network_node_files.append(filename)
    return network_node_files

network_node_result=get_network_node_files("/home/jovyan/work/neo4j/import/20-23/")
print(network_node_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_network_node.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_network_node.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_network_node.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_network_node.csv', 'AIA-201-225.ecar-last_network_node.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_network_node.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_network_node.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_network_node.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_network_node.csv', 'AIA-501-525.ecar-last_network_node.csv']


In [11]:
import os
#Gets all flow node files and puts them in a variable
def get_network_relations_files(directory):
    network_relations_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_network_relations.csv'):
             network_relations_files.append(filename)
    return network_relations_files

network_relations_result=get_network_relations_files("/home/jovyan/work/neo4j/import/20-23/")
print(network_relations_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_network_relations.csv', 'AIA-201-225.ecar-2019-12-07T19-16-05.788_network_relations.csv', 'AIA-201-225.ecar-2019-12-08T01-57-30.012_network_relations.csv', 'AIA-201-225.ecar-2019-12-08T05-46-21.658_network_relations.csv', 'AIA-201-225.ecar-last_network_relations.csv', 'AIA-501-525.ecar-2019-11-15T03-10-00.546_network_relations.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_network_relations.csv', 'AIA-501-525.ecar-2019-11-15T09-43-35.856_network_relations.csv', 'AIA-501-525.ecar-2019-11-15T13-29-59.064_network_relations.csv', 'AIA-501-525.ecar-last_network_relations.csv']


In [3]:
import os
#Gets all shell node files and puts them in a variable
def get_shell_node_files(directory):
    shell_node_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_shell_nodes.csv'):
             shell_node_files.append(filename)
    return shell_node_files

shell_node_result=get_shell_node_files("/home/jovyan/work/neo4j/import/20-23/")
print(shell_node_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_shell_nodes.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_shell_nodes.csv']


In [4]:
import os
#Gets all shell relationsfiles and puts them in a variable
def get_shell_relations_files(directory):
    shell_relations_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_shell_relations.csv'):
             shell_relations_files.append(filename)
    return shell_relations_files

shell_relations_result=get_shell_relations_files("/home/jovyan/work/neo4j/import/20-23/")
print(shell_relations_result)

['AIA-201-225.ecar-2019-12-07T22-06-33.589_shell_relations.csv', 'AIA-501-525.ecar-2019-11-15T05-59-37.208_shell_relations.csv']


**import data from files**

In [13]:
%%time

for filename in process_node_result:
    print(filename)
    
    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MERGE (c:Process {{
                ProcessGuid: row.ProcessGuid,
                ParentProcessGuid: row.ParentProcessGuid,
                Pid: row.Pid,
                Ppid: row.Ppid,
                EventTime: row.timestamp_str,
                Hostname: row.Hostname,
                Object: row.Object,
                Acuity_level: row.Acuity_level,
                Image_path: row.Image_path,
                Parent_image_path: row.Parent_image_path,
                User: row.User,
                Principal: row.Principal,
                CommandLine: row.CommandLine,
                CommandLine_length: row.CommandLine_length,
                Action: row.Action,
                Sid: row.Sid,
                Tid: row.Tid
            }})
        }} IN TRANSACTIONS OF 500 ROWS;
    '''.format(filename)
    gds.run_cypher(query)


AIA-201-225.ecar-2019-12-07T22-06-33.589_process_nodes.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_process_nodes.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_process_nodes.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_process_nodes.csv
AIA-201-225.ecar-last_process_nodes.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_process_nodes.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_process_nodes.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_process_nodes.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_process_nodes.csv
AIA-501-525.ecar-last_process_nodes.csv
CPU times: user 29 ms, sys: 2.82 ms, total: 31.8 ms
Wall time: 55.6 s


In [14]:
%%time

#Import and create the relations between the process parent and child relationship
for filename in process_relation_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MATCH (p1:Process {{ProcessGuid:row.ProcessGuid}})
            MATCH (p2:Process {{ProcessGuid:row.ParentProcessGuid}})
            MERGE (p1)-[r: Parent]->(p2)
        }}  IN TRANSACTIONS of 50 rows;
    '''.format(filename)
    gds.run_cypher(query)


AIA-201-225.ecar-2019-12-07T22-06-33.589_process_relations.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_process_relations.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_process_relations.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_process_relations.csv
AIA-201-225.ecar-last_process_relations.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_process_relations.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_process_relations.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_process_relations.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_process_relations.csv
AIA-501-525.ecar-last_process_relations.csv
CPU times: user 31.6 ms, sys: 1.95 ms, total: 33.6 ms
Wall time: 41.8 s


In [15]:
#Create constraint for network node, massivly speeds up import
gds.run_cypher("""CREATE CONSTRAINT Network_connection_node FOR (Network_connection_node:Network_connection_node) REQUIRE Network_connection_node.unknown IS UNIQUE""")

""


In [16]:
#Create index on parentprocessguid
gds.run_cypher("""CREATE TEXT INDEX quicker_import FOR (p:Process) ON (p.ParentProcessGuid)""")

""


In [17]:
%%time
#Import the network nodes

for filename in network_node_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MERGE (c:Network_connection_node {{
                unknown: row.unknown,
                ProcessGuid: row.ProcessGuid,
                EventTime: row.timestamp_str,
                Hostname: row.Hostname,
                Initiated: row.Initiated,
                SourceIp: row.SourceIp,
                SourcePort: row.SourcePort,
                DestinationIp: row.DestinationIp,
                DestinationPort: row.DestinationPort,
                Protocol: row.Protocol,
                ProcessID: row.Pid,
                Image_path: row.Image_path,
                User: row.Principal,
                Tid: row.Tid
            }})
        }} IN TRANSACTIONS OF 500 ROWS;
    '''.format(filename)
    gds.run_cypher(query)



AIA-201-225.ecar-2019-12-07T22-06-33.589_network_node.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_network_node.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_network_node.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_network_node.csv
AIA-201-225.ecar-last_network_node.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_network_node.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_network_node.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_network_node.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_network_node.csv
AIA-501-525.ecar-last_network_node.csv
CPU times: user 33.4 ms, sys: 5.58 ms, total: 39 ms
Wall time: 4min 19s


In [18]:
##Create text index on the data we are searching for
gds.run_cypher("""CREATE TEXT INDEX quicker_network_search FOR (c:Network_connection_node) ON (c.ProcessGuid)""")

""


In [19]:
%%time
#Import the network nodes relations 
for filename in network_relations_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MATCH (p1:Process {{ProcessGuid: row.ProcessGuid}})
            MATCH (c2:Network_connection_node {{ProcessGuid: row.ProcessGuid}})
            MERGE (p1)-[c: Connection]->(c2)
        }}  IN TRANSACTIONS of 40 rows;
    '''.format(filename)
    gds.run_cypher(query)


AIA-201-225.ecar-2019-12-07T22-06-33.589_network_relations.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_network_relations.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_network_relations.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_network_relations.csv
AIA-201-225.ecar-last_network_relations.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_network_relations.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_network_relations.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_network_relations.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_network_relations.csv
AIA-501-525.ecar-last_network_relations.csv
CPU times: user 2.25 s, sys: 1.01 s, total: 3.25 s
Wall time: 1d 3h 38s


In [20]:
#create constraint on files
gds.run_cypher("""CREATE CONSTRAINT File FOR (file:File) REQUIRE file.unknown IS UNIQUE""")

""


In [21]:
##Action,ParentProcessGuid,Hostname,unknown,Object,ProcessGuid,Pid,Ppid,Principal,Acuity_level,file_path,Image_path,info_class,Sid

In [22]:
%%time
#Import the file nodes
for filename in file_node_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MERGE (c:File {{
                unknown: row.unknown,
                Possible_ProcessGuid: row.Possible_ProcessGuid,
                ProcessGuid: row.ProcessGuid,
                EventTime: row.timestamp_str,
                Hostname: row.Hostname,
                ProcessID: row.Pid,
                Image_path: row.Image_path,
                User: row.Principal,
                Acuity_level: row.Acuity_level,
                file_path: row.file_path,
                info_class: row.info_class,
                Tid: row.Tid
            }})
        }} IN TRANSACTIONS OF 500 ROWS;
    '''.format(filename)
    gds.run_cypher(query)



AIA-201-225.ecar-2019-12-07T22-06-33.589_file_nodes.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_file_nodes.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_file_nodes.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_file_nodes.csv
AIA-201-225.ecar-last_file_nodes.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_file_nodes.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_file_nodes.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_file_nodes.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_file_nodes.csv
AIA-501-525.ecar-last_file_nodes.csv
CPU times: user 36.1 ms, sys: 6.97 ms, total: 43.1 ms
Wall time: 6min 54s


In [23]:
##Create text index on the data we are searching for
gds.run_cypher("""CREATE TEXT INDEX quicker_File_search FOR (f:File) ON (f.ProcessGuid)""")

""


In [24]:
%%time
#Import the file nodes relations
for filename in file_relations_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MATCH (p1:Process {{ProcessGuid: row.ProcessGuid}})
            MATCH (f1:File {{ProcessGuid: row.ProcessGuid}})
            MERGE (p1)-[c:Created_file]->(f1)
        }}  IN TRANSACTIONS of 40 rows;
    '''.format(filename)
    gds.run_cypher(query)


AIA-201-225.ecar-2019-12-07T22-06-33.589_file_relations.csv
AIA-201-225.ecar-2019-12-07T19-16-05.788_file_relations.csv
AIA-201-225.ecar-2019-12-08T01-57-30.012_file_relations.csv
AIA-201-225.ecar-2019-12-08T05-46-21.658_file_relations.csv
AIA-201-225.ecar-last_file_relations.csv
AIA-501-525.ecar-2019-11-15T03-10-00.546_file_relations.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_file_relations.csv
AIA-501-525.ecar-2019-11-15T09-43-35.856_file_relations.csv
AIA-501-525.ecar-2019-11-15T13-29-59.064_file_relations.csv
AIA-501-525.ecar-last_file_relations.csv
CPU times: user 9.34 s, sys: 4.88 s, total: 14.2 s
Wall time: 5d 2h 5min 10s


In [9]:
%%time
#Create constraint for shell node, massivly speeds up import
gds.run_cypher("""CREATE CONSTRAINT Shell FOR (shell:Shell) REQUIRE shell.unknown IS UNIQUE""")

CPU times: user 3.58 ms, sys: 1.18 ms, total: 4.76 ms
Wall time: 138 ms


""


In [6]:
%%time
#Import the file nodes
for filename in shell_node_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MERGE (s:Shell {{
                Action: row.Action,
                ProcessGuid: row.ProcessGuid,
                Hostname: row.Hostname,
                unknown: row.unknown,
                Object: row.Object,
                Unkown_ProcessGuid: row.Unkown_ProcessGuid,
                Pid: row.Pid,
                Ppid: row.Ppid,
                Principal: row.Principal,
                Acuity_level: row.Acuity_level,
                Image_path: row.Image_path,
                Sid: row.Sid,
                User: row.User,
                Tid: row.Tid,
                timestamp_str: row.timestamp_str,
                Severity_level: row.Severity_level,
                Host_type: row.Host_type,
                Console_Version: row.Console_Version,
                Host_ID: row.Host_ID,
                CommandLine: row.CommandLine,
                Engine_Version: row.Engine_Version,
                Runspace_ID: row.Runspace_ID,
                Pipeline_ID: row.Pipeline_ID,
                Command_Name: row.Command_Name,
                Command_Type: row.Command_Type,
                Script_Name: row.Script_Name,
                Command_Path: row.Command_Path,
                Sequence_Number: row.Sequence_Number,
                Shell_User: row.Shell_User,
                Connected_User: row.Connected_User,
                Shell_type: row.Shell_type  
            }})
        }} IN TRANSACTIONS OF 500 ROWS;
    '''.format(filename)
    gds.run_cypher(query)

AIA-201-225.ecar-2019-12-07T22-06-33.589_shell_nodes.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_shell_nodes.csv
CPU times: user 8.29 ms, sys: 0 ns, total: 8.29 ms
Wall time: 404 ms


In [13]:
##Create text index on the data we are searching for
gds.run_cypher("""CREATE TEXT INDEX quicker_shell_search FOR (s:Shell) ON (s.ProcessGuid)""")

""


In [7]:
%%time
#Import the file nodes relations
for filename in shell_relations_result:
    print(filename)

    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///20-23/{0}' AS row
        CALL {{
            WITH row
            MATCH (p1:Process {{ProcessGuid: row.ProcessGuid}})
            MATCH (s1:Shell {{ProcessGuid: row.ProcessGuid}})
            MERGE (p1)-[c:ran_shell_command]->(s1)
        }}  IN TRANSACTIONS of 40 rows;
    '''.format(filename)
    gds.run_cypher(query)

AIA-201-225.ecar-2019-12-07T22-06-33.589_shell_relations.csv
AIA-501-525.ecar-2019-11-15T05-59-37.208_shell_relations.csv
CPU times: user 8.59 ms, sys: 0 ns, total: 8.59 ms
Wall time: 429 ms


In [ ]:
# dont run what is¨after this frame, it is just troubleshooting stuff

In [ ]:
#Troubleshoot file relations 
#org file node import
%%time
#Import the file nodes
#for filename in file_node_result:
#    print(filename)

#    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///{0}' AS row
        CALL {{
            WITH row
            MERGE (c:File {{
                unknown: row.unknown,
                ParentProcessGuid: row.ParentProcessGuid,
                ProcessGuid: row.ProcessGuid,
                EventTime: row.timestamp_str,
                Hostname: row.Hostname,
                ProcessID: row.Pid,
                Image_path: row.Image_path,
                User: row.Principal,
                Acuity_level: row.Acuity_level,
                file_path: row.file_path,
                info_class: row.info_class,
                Tid: row.Tid
            }})
        }} IN TRANSACTIONS OF 500 ROWS;
 #   '''.format(filename)
 #   gds.run_cypher(query)


